# PA - Estatística - Pós Graduação CDS

### Preparação das bases de desenvolvimento e validação



## Imports:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from funcoes import formato_data, calcular_media_lat_lng


## Dataset:

### Tratando a tabela orders:

In [2]:
#lendo o arquivo:
orders = pd.read_csv('archive\olist_orders_dataset.csv')

#apenas order_status = delivered:
orders = orders[orders['order_status'] == 'delivered']

#selecionando as colunas:
orders = orders[['order_id', 'customer_id', 'order_approved_at', 
                 'order_delivered_customer_date', 'order_estimated_delivery_date']]

#excluindo as linhas nulas:
orders = orders.dropna()

#ajustando as colunas de data:
colunas_para_converter = ['order_approved_at', 
                          'order_delivered_customer_date', 
                          'order_estimated_delivery_date']

orders = formato_data(orders, colunas_para_converter)


### Tratando a tabela order_items:

In [3]:
#lendo o arquivo
order_items = pd.read_csv('archive\olist_order_items_dataset.csv')

#selecionando as colunas
order_items = order_items[['order_id', 'seller_id', 'product_id']]

### Tratando as tabelas customers, sellers e geolocation:

In [4]:
#lendo os arquivos
customers = pd.read_csv('archive\olist_customers_dataset.csv')
sellers = pd.read_csv('archive\olist_sellers_dataset.csv')
geolocation = pd.read_csv('archive\olist_geolocation_dataset.csv')

#selecionando as colunas:
customers = customers[['customer_id', 'customer_zip_code_prefix']]
sellers = sellers[['seller_id', 'seller_zip_code_prefix']]
geolocation = geolocation[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']]

In [5]:
#junções:

customers = customers.merge(geolocation, 
                left_on = 'customer_zip_code_prefix',
                right_on = 'geolocation_zip_code_prefix',
                how = 'left')

sellers = sellers.merge(geolocation, 
                left_on = 'seller_zip_code_prefix',
                right_on = 'geolocation_zip_code_prefix',
                how = 'left')

In [6]:
#Latitude média ('lat_media') e longitude média ('lng_media'):
customers = calcular_media_lat_lng(customers, 'customer_zip_code_prefix', 
                                     'geolocation_lat', 'geolocation_lng', 
                                     'customer_lat_media', 'customer_lgn_media')

sellers = calcular_media_lat_lng(sellers, 'seller_zip_code_prefix', 
                                     'geolocation_lat', 'geolocation_lng', 
                                     'seller_lat_media', 'seller_lgn_media')


In [7]:
#selecionando as colunas:
customers = customers[['customer_id','customer_lat_media', 'customer_lgn_media']]
sellers = sellers[['seller_id','seller_lat_media', 'seller_lgn_media']]

#excluindo linhas duplicadas:
customers = customers.drop_duplicates()
sellers = sellers.drop_duplicates()

### Tratando a tabela product:

In [8]:
#lendo o arquivo
product = pd.read_csv('archive\olist_products_dataset.csv')

#selecionando as colunas
product = product[['product_id', 'product_category_name', 
                 'product_weight_g','product_length_cm', 
                 'product_height_cm', 'product_width_cm']]

#criando a coluna de volume (cm³)
product['product_volume_cm3'] = (product['product_height_cm']*
                                 product['product_length_cm']*
                                 product['product_width_cm'])

### Fazendo as junções de orders, order_items, customers, sellers e products:

In [9]:
df = orders.merge(order_items, on='order_id', how='left') \
           .merge(customers, on='customer_id', how='left') \
           .merge(sellers, on='seller_id', how='left') \
           .merge(product, on= 'product_id', how = 'left')

#Excluindo linhas duplicadas e nulas:
df = df.drop_duplicates()
df.dropna(inplace = True)

### Criando novas colunas:

In [10]:
# Criando a coluna de distância euclidiana
df['dist_degrees'] = np.sqrt(
    (df['seller_lat_media'] - df['customer_lat_media'])**2 +
    (df['seller_lgn_media'] - df['customer_lgn_media'])**2
)

In [11]:
# Criando a coluna delta_time_dataset em dias, sendo a
# diferença de tempo entre a aprovação do pedido e a data estimada de entrega ao cliente:

df['delta_time_dataset'] = (df['order_estimated_delivery_date'] - df['order_approved_at']).dt.days

In [12]:
# Criando a coluna delta_time_real em dias, sendo a
# diferença de tempo entre a aprovação do pedido e a data real de entrega ao cliente:

df['delta_time_real'] = (df['order_delivered_customer_date'] - df['order_approved_at']).dt.days 


In [13]:
# Criando a coluna error_model_dataset em dias, sendo a
# diferença de tempo entre a data de entrega e a data estimada de entrega ao cliente:

df['error_model_dataset'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.days

### Selecionando as colunas:

In [14]:
base_final = df[['product_category_name','product_weight_g', 'product_volume_cm3', 
                 'dist_degrees', 'order_approved_at','order_delivered_customer_date',
                'delta_time_dataset', 'delta_time_real', 'error_model_dataset']]


In [15]:
#incluindo colunas de volume em litros e peso em quilos:

dados = base_final.copy() 

dados.loc[:, 'product_volume_l'] = dados['product_volume_cm3'] / 1000
dados.loc[:, 'product_weight_kg'] = dados['product_weight_g'] / 1000

### Dividindo em 70% e 30%:

In [16]:
desenvolvimento, validacao = train_test_split(dados, test_size=0.3, random_state=42)


### Salvando as bases:

In [17]:
dados.to_csv("base_completa.csv")
desenvolvimento.to_csv("base_desenvolvimento.csv")
validacao.to_csv("base_validacao.csv")